In [67]:
# **Import Libraries** 


In [68]:
import pandas as pd
import unicodeit
import matplotlib.pyplot as plt
import math
import seaborn as sns
from IPython.display import display,Markdown
import statsmodels.api as sm
import random
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score,root_mean_squared_error

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.layers import LSTM, Input, Dense, Concatenate, Flatten,Reshape,SimpleRNN
from keras_self_attention import SeqWeightedAttention
from keras.models import Model
import tensorflow as tf



In [69]:
# define random seeds
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

## **1. Load the "Individual household electric power consumption" dataset** 
***

In [70]:
# first define datatypes for the columns
dtypes = {'Date': str,
          'Time': str,
          'Global_active_power': float,
          'Global_reactive_power': float,
          'Voltage': float,
          'Global_intensity': float,
          'Sub_metering_1': float,
          'Sub_metering_2': float,
          'Sub_metering_3': float}

'''
fetch the dataset (acknowledging that the seperator is ';' and missing values are marked as '?'
'''

file_path = 'household_power_consumption.txt'
power_consumption_df = pd.read_csv(file_path, sep=';',dtype=dtypes,na_values='?')
power_consumption_df.head()

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

We can see that we have 9 features:
* Date
* Time
* Global_active_power
* Global_reactive_power
* Voltage
* Global_intensity
* Sub_metering_1
* Sub_metering_2
* Sub_metering_3

However, our target (active energy consumed every minute) is  not specified in the dataset. we will define "Global_active_power" as our target. Also, we would like to combine date and time to a one feature called: "datetime".

In [ ]:
# combine date and time into datetime
power_consumption_df['Datetime'] = pd.to_datetime(power_consumption_df['Date'] + ' ' + power_consumption_df['Time'], format='%d/%m/%Y %H:%M:%S')

# Set the datetime column as index
power_consumption_df.set_index('Datetime', inplace=True)

# Drop the original Date and TIme columns
power_consumption_df.drop(columns=['Date', 'Time'], inplace=True)
power_consumption_df.head()

## 2. **EDA**
***

In [ ]:
# Create a correlation plot to find the correlation between columns
plt.figure(figsize=(10, 10)) 
sns.heatmap(power_consumption_df.corr(method='pearson'),  
            annot=True,  
            cmap='RdYlGn',  
            fmt='.2f')  
plt.show()

In [ ]:
# Define colors for plotting
colors = sns.color_palette("Set2", len(power_consumption_df .columns))

# Define number of plots per row
plots_per_row = 5

# Calculate number of rows needed for the subplot grid
num_rows = len(power_consumption_df .columns) // plots_per_row + (len(power_consumption_df .columns) % plots_per_row > 0)

# Create subplots
fig, axes = plt.subplots(nrows=num_rows, ncols=plots_per_row, figsize=(20, 15))

# Flatten the axes array
axes = axes.flatten()

# Plot histograms for each column
for i, (col, color) in enumerate(zip(power_consumption_df .columns, colors)):
    power_consumption_df [col].hist(bins=50, ax=axes[i], color=color)
    axes[i].set_title(f"{col} Distribution")
    axes[i].set_xlabel(col)
    axes[i].set_ylabel("Frequency")

# Remove unused axes
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

# Adjust layout and display plot
plt.tight_layout()
plt.show()

From the distributions we can draw the following conclusions:
* The Global Active power seems to be consisted of two different normal distributions, one centered near 0, and the other centered at approximately 1.7. That could imply some different seasonal patterns within the year, one with lower power consumption, and the other - with a higher one.

* The Global reactive power seems to be less sparse, and will most likely be near 0. It implies that the electricity network is efficient and doesn't suffer from many losses :) .

* The voltage is clearly normally distributed with a mean around 240V (which makes sense since it's a French household). The voltage consumption is sparse, and that could be explained by different voltage consumptions from different appliances.

* Just like Global Active Power, the Global Intensity also seems to be consisted of two normal distributions which can imply different seasonal patterns - one centered at approximately 0 and one centered at approximately 5 .

* Sub_metering_1, Sub_metering_2, Sub_metering_3: High frequency of zero values indicates many periods with no sub-metered energy usage, with some occasional spikes.

### Interpretation: ###
***

* Global_active_power: Skewed to the right, indicating that most values are low, but there are occasional high values.
* Voltage: More normally distributed, showing stable voltage supply.
* Sub_metering_1, Sub_metering_2, Sub_metering_3: High frequency of zero values indicates many periods with no sub-metered energy usage, with some occasional spikes.

In [ ]:
def visualize_time_trends(data,columns_to_omit=None,resample=None):
    num_plots = len(data.columns) - len(columns_to_omit) if columns_to_omit is not None else len(data.columns)
    num_rows = num_cols = math.ceil(math.sqrt(num_plots))

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15,10))
    if columns_to_omit is  None:
        columns_to_omit = []

    i = j = 0

    for column in data.columns:
        if column not in columns_to_omit:
            if resample is None:
                axes[i, j].plot(data[column], label=column)

            else:
                axes[i, j].plot(data[column].resample(resample).mean(), label=column)

            axes[i, j].set_title(f'Time series for {column}')
            axes[i, j].set_xlabel('Datetime')
            axes[i, j].set_ylabel(f"value")
            axes[i,j].legend()
            j += 1

            if j == num_cols:
                j = 0
                i += 1

    if math.sqrt(num_plots) < num_cols:
        remainder = (num_cols ** 2) % num_plots
        for i in range(1, remainder + 1, 1):
            axes[-1,-1 * i].axis('off') # hide the last plots

    plt.tight_layout()
    plt.grid(True)
    plt.show()

visualize_time_trends(power_consumption_df)

According to the time series trends we can say the following:
1. **Global Active Power**

    * **Seasonality:** It seems like there are strong seasonal patterns with regular peaks and troughs (best appeared in the monthly resampling). This phenomenon could be explained by high power consumption during certain times of the year (for example, hot summer days)
         
   * **Cyclical Patterns:** There may be some longer-term trends, but they are overshadowed and difficult to isolate from the clear seasonal patterns without further analysis.

2. **Global reactive power**
    * **Seasonality:** Similar to Global Active Power, we can see a strong seasonal pattern (which makes sense, since the reactive power are the losses from the electrical appliances of the consumer, so they should have the same seasonality as the active power)
      
    * **Cyclical Patterns:** The same as the active power - longer term trends are less apparent, since there is a strong seasonal pattern.

3. **Voltage** 
    * **Seasonality:** The voltage shows less clear seasonal patterns (it doesn't seem to have a constant period) compared to power consumption, though there are still some periodic fluctuations (it seems clearer in the weekly or monthly resampling)
      
    * **Cyclical Patterns:** If there are any cyclical patterns in the voltage, they are not very prominent.

4. **Global Intensity**
    * **Seasonality:** There is a clear seasonal pattern, with regular fluctuations which again could be explained by periods of higher electricity demand.
      
    * **Cyclical Patterns:** Similar to the Global Active Power, the cyclical trends are less evident due to the dominant seasonal patterns.

5. **Sub Metering 1,2, and 3:** 
    * **Seasonality:** Each sub metering shows distinct seasonal patterns, likely corresponding to specific appliances that have different regular usage cycles.
      
    * **Cyclical Patterns:** Less apparent, as the data is dominated by strong seasonal patterns.



**Conclusion**

This dataset exhibits strong seasonal patterns across most of the features, as well as the target variable (Global_active_power). The seasonal patterns are consistent and predictable, and most likely correspond to higher and lower power demand during different periods of the year.

Cyclical patterns in this dataset are harder to distinguish from the seasonal ones, due to the dominance of the seasonal patterns.

### **Identify and handle missing values and outliers**
***

In [ ]:
display(power_consumption_df.isna().sum())
display(Markdown(f'##### {power_consumption_df.isna().sum().iloc[0]/ power_consumption_df.shape[0] * 100}% of the data is missing'))

In [ ]:
power_consumption_df_no_missing = power_consumption_df.copy()

for _column in power_consumption_df_no_missing.columns:
    # forward fill the column
    power_consumption_df_no_missing[_column] = power_consumption_df_no_missing[_column].ffill() 

# check for the existence of missing values again
power_consumption_df_no_missing.isna().sum()

Now that we have filled the missing values and re-calculated our target. let's deal with the outliers. let's make a seasonal decomposition (for example, let's do that to the global_active_power):

In [ ]:
power_consumption_df_no_missing = power_consumption_df.copy()

for _column in power_consumption_df_no_missing.columns:
    # forward fill the column
    power_consumption_df_no_missing[_column] = power_consumption_df_no_missing[_column].ffill() 

# check for the existence of missing values again
power_consumption_df_no_missing.isna().sum()

Now that we have filled the missing values and re-calculated our target. let's deal with the outliers. let's make a seasonal decomposition (for example, let's do that to the global_active_power):

In [ ]:
decomposition = sm.tsa.seasonal_decompose(power_consumption_df_no_missing['Global_active_power'],model='additive',period=365)
fig1 = decomposition.plot()
fig1.set_size_inches((15,10))
plt.show()

This decomposition consists of the following:
* **Observed:** The time series as it is (the actual data)
* **Trend:** reveals the long-term increase/decrease in the global_active_power consumption.
* **Seasonal:** The seasonal plot shows recurring patterns, for example: higher consumption during certain times of the month.
* **Residual:** The residual plot displays the noise or irregularities in the data (so we can use that in order to identify the outliers). We wil define an outlier as:
$outlier = resid.mean() \pm 3*resid.std()$
* We will check outliers for our target variable to see if it makes sense (for example - higher power consumption during summer). 

In [ ]:
def check_for_outliers(time_series,period,number_of_stds):
    seasonal_decomposition = sm.tsa.seasonal_decompose(time_series,model='additive',period=period)
    residuals = seasonal_decomposition.resid
    outliers_high = residuals[(residuals > residuals.mean() + number_of_stds * residuals.std())]
    outliers_low = residuals[(residuals < residuals.mean() - number_of_stds * residuals.std())]
    return outliers_low.index, outliers_high.index
(low_outliers_dates, high_outliers_dates) = check_for_outliers(time_series=power_consumption_df_no_missing['Global_active_power'],period=365,number_of_stds=3)
display(Markdown(f'##### low_power consumption dates are: {low_outliers_dates}'))
display(Markdown(f'##### high_power consumption dates are: {high_outliers_dates}'))

let's see what percentage of the data are these outliers before we make a decision:

In [ ]:
display(Markdown(f'##### {(len(low_outliers_dates) + len(high_outliers_dates)) / power_consumption_df_no_missing.shape[0] * 100}% of the data is considered outliers'))

since only 2% of the data is considered outliers, we will decide to drop them:


In [ ]:
power_consumption_df_no_missing_no_outliers = power_consumption_df_no_missing.copy()
outliers_to_drop = low_outliers_dates.union(high_outliers_dates)
power_consumption_df_no_missing_no_outliers.drop(outliers_to_drop,inplace=True)
display(power_consumption_df_no_missing_no_outliers)

In [71]:
energy= power_consumption_df_no_missing_no_outliers.copy



# Step 3: Implement a Linear Regression Model #
***


Split the data into training and testing sets


In [72]:
# Function to create lagged features
def create_lagged_features(df, columns, lags):
    for column in columns:
        for lag in range(1, lags + 1):
            df[f'{column}_lag_{lag}'] = df[column].shift(lag)
    return df.dropna()

# Function to split the data into train and test sets (80:20 ratio)
def split_data(df, target_column):
    train_size = int(len(df) * 0.8)
    train, test = df[:train_size], df[train_size:]
    X_train = train.drop(target_column, axis=1)
    y_train = train[target_column]
    X_test = test.drop(target_column, axis=1)
    y_test = test[target_column]
    return X_train, X_test, y_train, y_test

# Function to resample data
def resample_data(df, freq):
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("DataFrame index must be of datetime type")
    return df.resample(freq).mean()

# Function to prepare features and train linear regression model

# Function to prepare features and train linear regression model # 'minutely': 'min','hourly': 'h', 'daily': 'D','monthly': 'ME','weekly': 'W'
def train_and_evaluate_linear_regression(df, target_column, lags, resample_freq='D'):
    # Resample data
    resampled_df = resample_data(df, resample_freq)
    
    # Create lagged features
    columns_selected = ['Global_active_power']
    processed_df = create_lagged_features(resampled_df, columns_selected, lags)
    
    # Drop unwanted columns
    columns_to_drop = ['Global_reactive_power', 'Voltage', 'Global_intensity','Sub_metering_1','Sub_metering_2','Sub_metering_3']
    columns_to_drop = [col for col in columns_to_drop if col in processed_df.columns]
    processed_df = processed_df.drop(columns=columns_to_drop)
    
    # Split data
    X_train, X_test, y_train, y_test = split_data(processed_df, target_column)
    
    # Train linear regression model
    linear_model = LinearRegression()
    linear_model.fit(X_train, y_train)
    y_pred = linear_model.predict(X_test)
    
    # Evaluate model
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    return mse, mae, rmse, r2, y_test, y_pred


# Function to plot actual vs predicted values for different time periods
def plot_actual_vs_predicted(y_test, y_pred, model_name):
    time_periods = [50, 100, 200, 300]
    fig2, axes2 = plt.subplots(2, 2, figsize=(20, 14))
    axes2 = axes2.flatten()

    for i, period in enumerate(time_periods):
        axes2[i].plot(y_test.index[-period:], y_test[-period:], label='Actual', color='blue')
        axes2[i].plot(y_test.index[-period:], y_pred[-period:], label='Predicted', color='red', linestyle='dashed')
        axes2[i].set_title(f'Actual vs Predicted Global Active Power (Last {period} Data Points) - {model_name}')
        axes2[i].set_xlabel('DateTime')
        axes2[i].set_ylabel('Global Active Power (kilowatts)')
        axes2[i].legend()

    plt.tight_layout()
    plt.show()


In [ ]:
display(results_df)

we can see that the RNN model performed poorly comparing to the LinearRegression model.There could be some reasons for that:
* Lack of complex patterns - If the time series lacks intricate patterns or long-term dependencies, the complexity of an RNN may not offer any advantage over LinearRegression and could even hinder performance.
* Overfitting - The data is noisy (as we could see from the time series), Therefore a complex model (such as RNN) may lead to overfitting.
* Size of the dataset - LinearRegression will perform better on a smaller dataset, while RNN requires more data to learn effectively. Our dataset is large, but we resampled and perhaps under sampled it for RNN.
* Training Complexity - RNN is more challenging to train due to issues like vanishing/exploding gradients.
* Sensitivity to Hyperparameters - RNN is highly sensitive to hyperparameters (e.g. learning rate, sequence length, number of layers, hidden units). Poor tuning may lead the RNN to a bad performance, while LinearRegression model often require less hyperparameter tuning.
* Optimization Challenges - RNN may face optimization challenges that are less prominent in linear regression models.
* Feature Engineering - Linear Regression can benefit significantly from good feature engineering, which can sometimes be more intuitive. Creating lag variables (as we did) might make the linear model very powerful.

# LSTM #
***

In [ ]:
# Function to reshape data for LSTM input
def reshape_data_for_lstm(X_train, X_test):
    X_train_reshaped = np.reshape(X_train.values, (X_train.shape[0], 1, X_train.shape[1]))
    X_test_reshaped = np.reshape(X_test.values, (X_test.shape[0], 1, X_test.shape[1]))
    return X_train_reshaped, X_test_reshaped

# Function to prepare features and train LSTM model
def train_and_evaluate_lstm(df, target_column, lags, resample_freq='D', epochs=10, batch_size=32):
    # Ensure DataFrame index is datetime
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("DataFrame index must be of datetime type")

    # Resample data
    resampled_df = resample_data(df, resample_freq)
    
    # Create lagged features
    columns_selected = ['Global_active_power','Global_reactive_power', 'Voltage', 'Global_intensity','Sub_metering_1','Sub_metering_2','Sub_metering_3']
    processed_df = create_lagged_features(resampled_df, columns_selected, lags)
    
    # Drop unwanted columns
    columns_to_drop = ['Global_reactive_power', 'Voltage', 'Global_intensity','Sub_metering_1','Sub_metering_2','Sub_metering_3']
    columns_to_drop = [col for col in columns_to_drop if col in processed_df.columns]
    processed_df = processed_df.drop(columns=columns_to_drop)
    
    # Split data
    X_train, X_test, y_train, y_test = split_data(processed_df, target_column)
    
    # Reshape data for LSTM input
    X_train_reshaped, X_test_reshaped = reshape_data_for_lstm(X_train, X_test)
    
    # Build LSTM model
    lstm_model = Sequential()
    lstm_model.add(Input(shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2])))
    lstm_model.add(LSTM(50))
    lstm_model.add(Dense(1))
    lstm_model.compile(optimizer='adam', loss='mse')
    
    # Train LSTM model
    lstm_model.fit(X_train_reshaped, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    
    # Make predictions
    y_pred = lstm_model.predict(X_test_reshaped)
    
    # Evaluate model
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    return mse, mae, rmse, r2, y_test, y_pred



In [ ]:
# Define parameters
target_column = 'Global_active_power'
lags = 3
resample_freq = ('min')  # Change this to 'T', 'H', 'D', 'W', 'min' as needed

# Train and evaluate LSTM model
mse, mae, rmse, r2, y_test, y_pred = train_and_evaluate_lstm(power_consumption_df_no_missing_no_outliers, target_column, lags, resample_freq)
# Print results
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R²: {r2}')

plot_actual_vs_predicted(y_test, y_pred, 'LSTM')
results_df = pd.concat([results_df,pd.DataFrame({"Model_Name": "LSTM", "MSE": mse, "MAE": mae, "RMSE":rmse, unicodeit.replace('R^2'): r2},index=[0])],axis=0, ignore_index=True)

### **Compare Results**
***

In [ ]:
display(results_df)

The RNN and the LSTM are very similar, and the Linear regression still wins.

# LSTM with Attention #
***

In [ ]:


# Function to build and train an LSTM model with Attention
def train_and_evaluate_attention_lstm(df, target_column, lags, resample_freq='D', epochs=10, batch_size=32):
    # Resample data
    resampled_df = resample_data(df, resample_freq)
    
    # Create lagged features
    columns_selected = ['Global_active_power']
    processed_df = create_lagged_features(resampled_df, columns_selected, lags)
    
    # Drop unwanted columns
    columns_to_drop = ['Global_reactive_power', 'Voltage', 'Global_intensity', 'Sub_metering_1', 'Sub_metering_2', 'Sub_metering_3']
    columns_to_drop = [col for col in columns_to_drop if col in processed_df.columns]
    processed_df = processed_df.drop(columns=columns_to_drop)
    
    # Split data
    X_train, X_test, y_train, y_test = split_data(processed_df, target_column)
    
    # Reshape data for LSTM input
    X_train_reshaped, X_test_reshaped = reshape_data_for_rnn(X_train, X_test)
    
    # Build LSTM model with Attention
    input_layer = Input(shape=(X_train_reshaped.shape[1], X_train_reshaped.shape[2]))
    lstm_out, state_h, state_c = LSTM(50, return_sequences=True, return_state=True)(input_layer)
    
    attention = SeqWeightedAttention()(lstm_out)
    # Reshape lstm_out to match the attention output shape
    attention_reshaped = Reshape((1, 50))(attention)
    context_vector = Concatenate(axis=-1)([attention_reshaped, lstm_out])
    context_vector = Flatten()(context_vector)
    output = Dense(1)(context_vector)
    attention_lstm_model = Model(inputs=input_layer, outputs=output)
    attention_lstm_model.compile(optimizer='adam', loss='mse')
    
    # Train LSTM model with Attention
    attention_lstm_model.fit(X_train_reshaped, y_train, epochs=epochs, batch_size=batch_size, verbose=0)
    
    # Make predictions
    y_pred = attention_lstm_model.predict(X_test_reshaped)
    
    # Evaluate model
    mse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    
    return mse, mae, rmse, r2, y_test, y_pred



In [ ]:
# Define parameters
target_column = 'Global_active_power'
lags = 3
resample_freq = ('h')  # Change this to 'T', 'H', 'D', 'W', 'min' as needed


mse_att_lstm, mae, rmse, r2, y_test, y_pred = train_and_evaluate_attention_lstm(power_consumption_df_no_missing_no_outliers, target_column, lags, resample_freq)

# Print results
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'RMSE: {rmse}')
print(f'R²: {r2}')
plot_actual_vs_predicted(y_test, y_pred, 'Attention LSTM')
results_df = pd.concat([results_df,pd.DataFrame({"Model_Name": "LSTM with attention", "MSE": mse, "MAE": mae, "RMSE":rmse, unicodeit.replace('R^2'): r2},index=[0])],axis=0, ignore_index=True)

### **Compare Results**
***

In [ ]:
display(results_df)

It Doesn't seem like the attention layer had a meaningful effect to the LSTM model. The linear regression model still wins.

## Data Augmentation  ##
***

In [ ]:
# Function to augment data
def augment_data(df, max_change=0.1):
    augmented_df = df.copy()
    num_rows_to_modify = int(len(df) * max_change)
    rows_to_modify = random.sample(range(len(df)), num_rows_to_modify)
    
    for row in rows_to_modify:
        augmented_df.iloc[row] *= random.uniform(0.9, 1.1)  # Apply random change within ±10%
    
    return augmented_df

def experiment_with_augmentation(df, target_column, lags, resample_freq, max_change=0.1, epochs=10, batch_size=32):
    augmented_df = augment_data(df, max_change)
    
    # Train and evaluate Attention LSTM model with augmented data
    attention_lstm_mse, attention_lstm_mae, attention_lstm_rmse, attention_lstm_r2, y_test_attention_lstm_aug, y_pred_attention_lstm_aug = train_and_evaluate_attention_lstm(augmented_df, target_column, lags, resample_freq, epochs, batch_size)
    
    # Train and evaluate LSTM model with augmented data
    lstm_mse, lstm_mae, lstm_rmse, lstm_r2, y_test_lstm_aug, y_pred_lstm_aug = train_and_evaluate_lstm(augmented_df, target_column, lags, resample_freq, epochs, batch_size)
    
    # Train and evaluate RNN model with augmented data
    rnn_mse, rnn_mae, rnn_rmse, rnn_r2, y_test_rnn_aug, y_pred_rnn_aug = train_and_evaluate_rnn(augmented_df, target_column, lags, resample_freq, epochs, batch_size)
    
    return {
        "aug_attention_lstm": (attention_lstm_mse, attention_lstm_mae, attention_lstm_rmse, attention_lstm_r2, y_test_attention_lstm_aug, y_pred_attention_lstm_aug),
        "aug_lstm": (lstm_mse, lstm_mae, lstm_rmse, lstm_r2, y_test_lstm_aug, y_pred_lstm_aug),
        "aug_rnn": (rnn_mse, rnn_mae, rnn_rmse, rnn_r2, y_test_rnn_aug, y_pred_rnn_aug)
    }

## Data Reduction  ##
***

In [ ]:
def reduce_data(df, reduction_percentage=0.1):
    reduced_df = df.sample(frac=(1 - reduction_percentage), random_state=42).sort_index()
    return reduced_df

def experiment_with_reduction(df, target_column, lags, resample_freq, reduction_percentage=0.1, epochs=10, batch_size=32):
    reduced_df = reduce_data(df, reduction_percentage)
    
    # Train and evaluate Attention LSTM model with reduced data
    attention_lstm_mse, attention_lstm_mae, attention_lstm_rmse, attention_lstm_r2, y_test_attention_lstm_red, y_pred_attention_lstm_red = train_and_evaluate_attention_lstm(reduced_df, target_column, lags, resample_freq, epochs, batch_size)
    
    # Train and evaluate LSTM model with reduced data
    lstm_mse, lstm_mae, lstm_rmse, lstm_r2, y_test_lstm_red, y_pred_lstm_red = train_and_evaluate_lstm(reduced_df, target_column, lags, resample_freq, epochs, batch_size)
    
    # Train and evaluate RNN model with reduced data
    rnn_mse, rnn_mae, rnn_rmse, rnn_r2, y_test_rnn_red, y_pred_rnn_red = train_and_evaluate_rnn(reduced_df, target_column, lags, resample_freq, epochs, batch_size)
    
    return {
        "red_attention_lstm": (attention_lstm_mse, attention_lstm_mae, attention_lstm_rmse, attention_lstm_r2, y_test_attention_lstm_red, y_pred_attention_lstm_red),
        "red_lstm": (lstm_mse, lstm_mae, lstm_rmse, lstm_r2, y_test_lstm_red, y_pred_lstm_red),
        "red_rnn": (rnn_mse, rnn_mae, rnn_rmse, rnn_r2, y_test_rnn_red, y_pred_rnn_red)
    }


## Run Models With Data Augmentation Experiment ##
***

In [ ]:
import pandas as pd

# Define parameters
target_column = 'Global_active_power'
lags = 3
resample_freq = 'h'
max_change = 0.1
epochs = 10  # You need to define epochs and batch_size for the function calls
batch_size = 32

# Original model training and evaluation
attention_lstm_mse, attention_lstm_mae, attention_lstm_rmse, attention_lstm_r2, y_test_attention_lstm, y_pred_attention_lstm = train_and_evaluate_attention_lstm(power_consumption_df_no_missing_no_outliers, target_column, lags, resample_freq, epochs, batch_size)

lstm_mse, lstm_mae, lstm_rmse, lstm_r2, y_test_lstm, y_pred_lstm = train_and_evaluate_lstm(power_consumption_df_no_missing_no_outliers, target_column, lags, resample_freq, epochs, batch_size)

rnn_mse, rnn_mae, rnn_rmse, rnn_r2, y_test_rnn, y_pred_rnn = train_and_evaluate_rnn(power_consumption_df_no_missing_no_outliers, target_column, lags, resample_freq, epochs, batch_size)

# Data augmentation experiment
aug_results = experiment_with_augmentation(power_consumption_df_no_missing_no_outliers, target_column, lags, resample_freq, max_change, epochs, batch_size)

# Update results DataFrame
results_df = pd.DataFrame(columns=["Model_Name", "MSE", "MAE", "RMSE", 'R^2'])

# Original models
results_df = pd.concat([results_df, pd.DataFrame({
    "Model_Name": ["Attention LSTM", "LSTM", "RNN"], 
    "MSE": [attention_lstm_mse, lstm_mse, rnn_mse], 
    "MAE": [attention_lstm_mae, lstm_mae, rnn_mae], 
    "RMSE": [attention_lstm_rmse, lstm_rmse, rnn_rmse], 
    'R^2': [attention_lstm_r2, lstm_r2, rnn_r2]
})], axis=0, ignore_index=True)

# Augmented models
results_df = pd.concat([results_df, pd.DataFrame({
    "Model_Name": ["Attention LSTM Augmented", "LSTM Augmented", "RNN Augmented"], 
    "MSE": [aug_results["aug_attention_lstm"][0], aug_results["aug_lstm"][0], aug_results["aug_rnn"][0]], 
    "MAE": [aug_results["aug_attention_lstm"][1], aug_results["aug_lstm"][1], aug_results["aug_rnn"][1]], 
    "RMSE": [aug_results["aug_attention_lstm"][2], aug_results["aug_lstm"][2], aug_results["aug_rnn"][2]], 
    'R^2': [aug_results["aug_attention_lstm"][3], aug_results["aug_lstm"][3], aug_results["aug_rnn"][3]]
})], axis=0, ignore_index=True)

# Display results
display(results_df)


## Run Models With Data reduction Experiment ##
***

In [ ]:

# Experiment with reduced data
red_results = experiment_with_reduction(power_consumption_df_no_missing_no_outliers, target_column, lags, resample_freq)# Train and evaluate Attention LSTM model with reduced data


# Update results DataFrame
results_df = pd.DataFrame(columns=["Model_Name", "MSE", "MAE", "RMSE", 'R^2'])

# Original models
results_df = pd.concat([results_df, pd.DataFrame({
    "Model_Name": ["Attention LSTM", "LSTM", "RNN"], 
    "MSE": [attention_lstm_mse, lstm_mse, rnn_mse], 
    "MAE": [attention_lstm_mae, lstm_mae, rnn_mae], 
    "RMSE": [attention_lstm_rmse, lstm_rmse, rnn_rmse], 
    'R^2': [attention_lstm_r2, lstm_r2, rnn_r2]
})], axis=0, ignore_index=True)

# Augmented models
results_df = pd.concat([results_df, pd.DataFrame({
    "Model_Name": ["Attention LSTM Augmented", "LSTM Augmented", "RNN Augmented"], 
    "MSE": [aug_results["aug_attention_lstm"][0], aug_results["aug_lstm"][0], aug_results["aug_rnn"][0]], 
    "MAE": [aug_results["aug_attention_lstm"][1], aug_results["aug_lstm"][1], aug_results["aug_rnn"][1]], 
    "RMSE": [aug_results["aug_attention_lstm"][2], aug_results["aug_lstm"][2], aug_results["aug_rnn"][2]], 
    'R^2': [aug_results["aug_attention_lstm"][3], aug_results["aug_lstm"][3], aug_results["aug_rnn"][3]]
})], axis=0, ignore_index=True)

# Reduced models
results_df = pd.concat([results_df, pd.DataFrame({
    "Model_Name": ["Attention LSTM Reduced", "LSTM Reduced", "RNN Reduced"], 
    "MSE": [red_results["red_attention_lstm"][0], red_results["red_lstm"][0], red_results["red_rnn"][0]], 
    "MAE": [red_results["red_attention_lstm"][1], red_results["red_lstm"][1], red_results["red_rnn"][1]], 
    "RMSE": [red_results["red_attention_lstm"][2], red_results["red_lstm"][2], red_results["red_rnn"][2]], 
    'R^2': [red_results["red_attention_lstm"][3], red_results["red_lstm"][3], red_results["red_rnn"][3]]
})], axis=0, ignore_index=True)

# Display results
display(results_df)


In [ ]:
# Define parameters
target_column = 'Global_active_power'
lags = 3
resample_freq = '2h'

# Original model training and evaluation
attention_lstm_mse, attention_lstm_mae, attention_lstm_rmse, attention_lstm_r2, y_test_attention_lstm, y_pred_attention_lstm = train_and_evaluate_attention_lstm(power_consumption_df_no_missing_no_outliers, target_column, lags, resample_freq)

lstm_mse, lstm_mae, lstm_rmse, lstm_r2, y_test_lstm, y_pred_lstm = train_and_evaluate_lstm(power_consumption_df_no_missing_no_outliers, target_column, lags, resample_freq)

rnn_mse, rnn_mae, rnn_rmse, rnn_r2, y_test_rnn, y_pred_rnn = train_and_evaluate_rnn(power_consumption_df_no_missing_no_outliers, target_column, lags, resample_freq )

In [ ]:
# Original models
results_df = pd.concat([results_df, pd.DataFrame({
    "Model_Name": ["Attention LSTM", "LSTM", "RNN"], 
    "MSE": [attention_lstm_mse, lstm_mse, rnn_mse], 
    "MAE": [attention_lstm_mae, lstm_mae, rnn_mae], 
    "RMSE": [attention_lstm_rmse, lstm_rmse, rnn_rmse], 
    'R^2': [attention_lstm_r2, lstm_r2, rnn_r2]
})], axis=0, ignore_index=True)

display(results_df)